In [44]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

# import warnings
# warnings.filterwarnings("ignore", category=DeprecationWarning);

import sys
sys.executable

'/home/mingrui/anaconda3/envs/py36_cu90/bin/python'

In [45]:
import os
import glob
import shutil
from random import shuffle

# plotting
import PIL
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import Image
plt.rcParams['figure.figsize'] = [15, 15]

import deephistopath.wsi.slide as slide

import py_wsi
import py_wsi.imagepy_toolkit as tk

from wsi_utils import multiprocessing_pipeline, file_stats, turtle_stats

import pandas as pd
pd.set_option('display.max_columns',None)

# Setup multi classes

# Folder structure

/media/disk2/PathologyNet/  
|-- TCGA_Glioma  
|-- Meningiomas  
|-- TCGA_Lung  
|-- ...  


In [46]:
tumor_folder_names = [['TCGA_Glioma'], ['Meningiomas', 'CGGA_Meningioma'], ['TCGA_Lung'], ['TCGA_Kidney'], 
                      ['TCGA_Breast'], ['TCGA_Uterus'], ['TCGA_Colorectal'], 
                      ['Ependymoma_Craniopharyngioma', 'CGGA_EC'], ['TCGA_WSI_glioma', 'TCGA_Glioma_Frozen'],
                      ['TiantanImage/IDH_bingli', 'CGGA_Glioma']]
tumor_type_list = []

SYNOLOGY_DIR = '/media/pathimg/'
PATH_NET_DIR = '/media/disk2/PathologyNet'
#PATH_NET_DIR = './data/'

for tumor in tumor_folder_names:
    # use first as origin folder name
    tumor_dir = os.path.join(SYNOLOGY_DIR, tumor[0])
    # use last as destination folder name
    db_dir = os.path.join(PATH_NET_DIR, tumor[-1])
    if os.path.exists(db_dir):
        shutil.rmtree(db_dir)
    os.mkdir(db_dir)
    # name db with destination folder name
    db_name = tumor[-1] + '_patch_db'
    tumor_type_list.append([tumor_dir, db_dir, db_name])
    
print(tumor_type_list)

[['/media/pathimg/TCGA_Glioma', '/media/disk2/PathologyNet/TCGA_Glioma', 'TCGA_Glioma_patch_db'], ['/media/pathimg/Meningiomas', '/media/disk2/PathologyNet/CGGA_Meningioma', 'CGGA_Meningioma_patch_db'], ['/media/pathimg/TCGA_Lung', '/media/disk2/PathologyNet/TCGA_Lung', 'TCGA_Lung_patch_db'], ['/media/pathimg/TCGA_Kidney', '/media/disk2/PathologyNet/TCGA_Kidney', 'TCGA_Kidney_patch_db'], ['/media/pathimg/TCGA_Breast', '/media/disk2/PathologyNet/TCGA_Breast', 'TCGA_Breast_patch_db'], ['/media/pathimg/TCGA_Uterus', '/media/disk2/PathologyNet/TCGA_Uterus', 'TCGA_Uterus_patch_db'], ['/media/pathimg/TCGA_Colorectal', '/media/disk2/PathologyNet/TCGA_Colorectal', 'TCGA_Colorectal_patch_db'], ['/media/pathimg/Ependymoma_Craniopharyngioma', '/media/disk2/PathologyNet/CGGA_EC', 'CGGA_EC_patch_db'], ['/media/pathimg/TCGA_WSI_glioma', '/media/disk2/PathologyNet/TCGA_Glioma_Frozen', 'TCGA_Glioma_Frozen_patch_db'], ['/media/pathimg/TiantanImage/IDH_bingli', '/media/disk2/PathologyNet/CGGA_Glioma', '

In [47]:
def process_tumor_type(tumor_dir, db_dir, db_name, num_to_process):
    turtle = py_wsi.Turtle(tumor_dir, db_dir, db_name)
    turtle_stats(turtle)
#     file_stats(turtle.files_path[0])
    files_path = turtle.files_path[:num_to_process]
    files_name = turtle.files[:num_to_process]
    
    return files_path, files_name

In [58]:
# %%capture capt
for tumor_type in tumor_type_list:
    files_path, files_name = process_tumor_type(tumor_type[0], tumor_type[1], tumor_type[2], 10)
    db_dir = tumor_type[1]
    # create db dir if doesn't exist at destination
    if os.path.exists(db_dir):
        shutil.rmtree(db_dir)
    os.mkdir(db_dir)
    multiprocessing_pipeline(tumor_type[0], db_dir, files_path, files_name, tile_size=512, objective_power=20, num_tiles=50)
    break

1704 WSI found in directory.


Total WSI images: 1704
LMDB name: TCGA_Glioma_patch_db
file names: ['TCGA-06-0171-01Z-00-DX3.svs' 'TCGA-R8-A73M-01Z-00-DX1.svs'
 'TCGA-QH-A6X8-01Z-00-DX1.svs' 'TCGA-QH-A65X-01Z-00-DX1.svs'
 'TCGA-02-0009-01Z-00-DX3.svs']
file paths: ['/media/pathimg/TCGA_Glioma/TCGA-06-0171-01Z-00-DX3.svs'
 '/media/pathimg/TCGA_Glioma/TCGA-R8-A73M-01Z-00-DX1.svs'
 '/media/pathimg/TCGA_Glioma/TCGA-QH-A6X8-01Z-00-DX1.svs'
 '/media/pathimg/TCGA_Glioma/TCGA-QH-A65X-01Z-00-DX1.svs'
 '/media/pathimg/TCGA_Glioma/TCGA-02-0009-01Z-00-DX3.svs']
/media/pathimg/TCGA_Glioma
/media/pathimg/TCGA_Glioma/TCGA-06-0171-01Z-00-DX3.svs

Opening Slide #1: /media/pathimg/TCGA_Glioma/TCGA-06-0171-01Z-00-DX3.svs
Level count: 3
Level dimensions: ((46047, 41602), (11511, 10400), (2877, 2600))
Level downsamples: (1.0, 4.000226463984283, 16.00299149755354)
Dimensions: (46047, 41602)
Objective power: 20
Associated images:
  label: <PIL.Image.Image image mode=RGBA size=456x472 at 0x7F6210CFF080>
  macr

# Move files to training folders

In [49]:
def copy_tiles(slides, dest, start, end):
    print('copy to', dest)
    for slide in slides[start:end]:
        tiles = glob.glob(os.path.join(slide, '*'))
        for tile in tiles:
            shutil.copy(tile, dest)